In [40]:
import numpy as np
from pandas import DataFrame
import pandas as pd
import scipy as sp
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn.svm import SVC
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn import cross_validation

In [41]:
np.set_printoptions(precision=5,suppress=True)


In [42]:
#def read_data(folder, name1, name2, name3):
#    loc = "%s/%s.txt" %(folder, name1)
#    train_set = pd.read_csv(loc, sep="\t")
#    print "Training set has %s rows and %s coumns" %(train_set.shape[0], train_set.shape[1])
#    loc = "%s/%s.txt" %(folder, name2)
#    test_set = pd.read_csv(loc, sep="\t")
#    print "Testing set has %s rows and %s coumns" %(test_set.shape[0], test_set.shape[1])
#    loc = "%s/%s.txt" %(folder, name3)
#    val_set = pd.read_csv(loc, sep="\t")
#    print "Validation set has %s rows and %s coumns" %(val_set.shape[0], val_set.shape[1])
#    return train_set, test_set, val_set

In [43]:
dataset = sp.genfromtxt("data3/cleaned.txt", delimiter="\t")
#np.isnan(dataset)

In [44]:
dataset.shape

(1269L, 27L)

In [45]:
#train_set, test_set, val_set = read_data("data3", "train_data", "test_data", "valid_data")

In [46]:
dataset = np.delete(dataset,[0,26],1)
dataset.shape

(1269L, 25L)

In [47]:
print dataset[3]

[  52.       2.       8.3      1.31     3.08     0.425    0.       0.       0.
    0.5      0.87     0.578    0.6      1.25     1.85     0.27     0.17
    0.35     1.12     0.42     3.12     2.       2.     104.     208.   ]


In [48]:
dataset = dataset[~np.isnan(dataset).any(axis=1)]
dataset.shape

(1268L, 25L)

In [49]:
columns = {"GP":0, "GS":1, "MIN":2, "FGM":3,"FGA":4,"FG%":5,"3PM":6,"3PA":7,"3P%":8,"FTM":9,"FTA":10,"FT%":11,"OFF":12,"DEF":13,
              "TRB":14,"AST":15,"STL":16,"BLK":17,"PF":18,"TOV":19,"PTS":20,"YR":21,"W":22,"H":23} 

# Labels

In [50]:
def np_labeliser(data,col):
    labels = data[:,col]
    return labels

In [51]:
labels = np_labeliser(dataset,22)
labels[:10]

array([ 2.,  1.,  2.,  2.,  2.,  3.,  2.,  2.,  3.,  1.])

# Feature Selection

In [52]:
def np_featuriser(dataset, feature_list):
    
    features = np.delete(dataset,feature_list,1)
    #test = np.delete(test,feature_list,1)
    #val = np.delete(val,feature_list,1)
    
    return features

In [53]:
feature_list = [22]
#np.set_printoptions(precision=4)
print len(dataset[0])
#train_features_nb, test_features_nb, val_features_nb = np_featuriser(train_set_nb, test_set_nb, val_set_nb, feature_list)
features = np_featuriser(dataset, feature_list)
print len(features[0])

25
24


In [54]:
def vt_fsel(feature_set):
    sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
    sel.fit_transform(feature_set)
    vt_list = sel.get_support()
    l_vt = []
    j = 0
    for i in vt_list:
        if i == False:
            l_vt.append(j)
            print "%s. feature name: %s" %(j, columns.keys()[columns.values().index(j)])
        j = j+1
    return l_vt    

In [55]:
list_vt = vt_fsel(features)

5. feature name: FG%
8. feature name: 3P%
11. feature name: FT%


In [56]:
features_vt = np_featuriser(features, list_vt)

features_vt.shape

(1268L, 21L)

In [57]:
def sup_features(usp_list,x):
    remove = []
    j = 0
    for i in usp_list:
        if i == False:
            remove.append(j)
            if x=="vt":
                print "%s. feature name: %s" %(j, columns.keys()[columns.values().index(j)])
        elif x == "uni":
            print "%s. feature name: %s" %(j, columns.keys()[columns.values().index(j)])
        j = j+1  

    return remove
#sup_features(support, "uni")

In [58]:
def feature_selection(clf, features, labels, domain):
    none = features
    #print none[0]
    domain = np_featuriser(features, domain)
    #print domain[0]
    clf = Pipeline([('feature_selection',SelectPercentile(f_classif, percentile=20)),
  ('classification', clf)])
    clf.fit(features, labels)
    print "\nUnivariate - valuable features \n"
    uni = sup_features(clf.named_steps['feature_selection'].get_support(),"uni")
    uni = np_featuriser(features, uni)
    #print uni[0]
    clf = Pipeline([('feature_selection',VarianceThreshold(threshold=(.8 * (1 - .8)))),
  ('classification', clf)])
    clf.fit(features, labels)
    print "\nVariance Threshold - removed \n"
    v_th = sup_features(clf.named_steps['feature_selection'].get_support(), "vt")
    #print v_th[0]
    v_th = np_featuriser(features, v_th)
    return none, domain, uni, v_th  
#clf = GaussianNB()
#svm = SVC()
#svm.set_params(kernel='linear')
#clf = Pipeline([('feature_selection',VarianceThreshold(threshold=(.8 * (1 - .8)))),
 # ('classification', svm)])
#clf.fit(features, labels)
#support = clf.named_steps['feature_selection'].get_support()
#print support
#p = clf.predict(features)
#acc = metrics.accuracy_score(labels,p)
#conf = metrics.confusion_matrix(labels, p)
#print acc
#print conf
domain = [columns["GP"],columns["GS"],columns["MIN"],columns["FG%"],
     columns["3P%"],columns["FT%"],columns["PTS"],columns["YR"],columns['3PM'],columns['FTM'],columns['FGM']]
#none, domain, uni, vth = feature_selection(clf, features, labels, domain)
#print none.shape,domain.shape,uni.shape,vth.shape

In [59]:
def cross_val(clf, f, l, name):
    print "\nFeature selection: %s" %name
    scores = cross_validation.cross_val_score(clf, f, l, cv=10)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    

In [60]:
def clf_all(clf, features, labels, domain):
    none, domain, uni, vth = feature_selection(clf, features, labels, domain)
    
    cross_val(clf, none, labels, "None")
    print "Number of features left: %s" %none.shape[1]
    cross_val(clf, domain, labels, "Domain")
    print "Number of features left: %s" %domain.shape[1]
    cross_val(clf, uni, labels, "Univariate")
    print "Number of features left: %s" %uni.shape[1]
    cross_val(clf, vth, labels, "Variance Threshold")
    print "Number of features left: %s" %vth.shape[1]

# ALL Results

In [61]:
#def print_metrics(name, accuracy, conf_matrix):
#    print "Feature selection: %s\n" %name
#   print "Accuracy score: %s\n" %accuracy
 #   print "Confusion matrix:"
#  print "\n%s" %conf_matrix
  # print"\n"
    
#def clf(clf, tr, tr_labels, val, val_labels):
#        clf.fit(tr, tr_labels)
#        
#        pred = clf.predict(val)
#        
#        acc = metrics.accuracy_score(val_labels,pred)
#        conf = metrics.confusion_matrix(val_labels, pred)
        
#        return acc, conf

In [62]:
#def clf_all(CLF, l_none, l_domain, l_uni, l_vt, train_all, test_all, val_all):
#    tr_none, ts_none, val_none = np_featuriser(train_all, test_all, val_all, l_none)
#    print tr_none.shape
#    tr_domain, ts_domain, val_domain = np_featuriser(train_all, test_all, val_all, l_domain)
#    
#    tr_uni, ts_uni, val_uni = np_featuriser(train_all, test_all, val_all, l_uni)
#    
#    tr_vt, ts_vt, val_vt = np_featuriser(train_all, test_all, val_all, l_vt)
    
    #clfnb = GaussianNB()
    #print "Naive Bayes\n"
    
#    acc_none, conf_none = clf(CLF, tr_none, train_labels, val_none, test_labels)
#    print_metrics("None", acc_none, conf_none)
    
#    acc_domain, conf_domain = clf(CLF, tr_domain, train_labels, val_domain, val_labels)
#    print_metrics("Domain knowledge", acc_domain, conf_domain)

#    acc_uni, conf_uni = clf(CLF, tr_uni, train_labels, val_uni, val_labels)
#    print_metrics("Univariate", acc_uni, conf_uni)
    
#    acc_vt, conf_vt = clf(CLF, tr_vt, train_labels, val_vt, val_labels)
#    print_metrics("Variance Threshold", acc_vt, conf_vt)

# Naive Bayes

In [63]:
clf_all(GaussianNB(), features, labels, domain) 


Univariate - valuable features 

12. feature name: OFF
15. feature name: AST
17. feature name: BLK
22. feature name: W
23. feature name: H

Variance Threshold - removed 

5. feature name: FG%
8. feature name: 3P%
11. feature name: FT%

Feature selection: None
Accuracy: 0.81 (+/- 0.08)
Number of features left: 24

Feature selection: Domain
Accuracy: 0.87 (+/- 0.07)
Number of features left: 13

Feature selection: Univariate
Accuracy: 0.89 (+/- 0.04)
Number of features left: 5

Feature selection: Variance Threshold
Accuracy: 0.82 (+/- 0.07)
Number of features left: 21


# SVM

In [64]:
svm = SVC()
svm = svm.set_params(kernel='linear')
clf_all(svm, features, labels, domain) 


Univariate - valuable features 

12. feature name: OFF
15. feature name: AST
17. feature name: BLK
22. feature name: W
23. feature name: H

Variance Threshold - removed 

5. feature name: FG%
8. feature name: 3P%
11. feature name: FT%

Feature selection: None
Accuracy: 0.90 (+/- 0.04)
Number of features left: 24

Feature selection: Domain
Accuracy: 0.91 (+/- 0.05)
Number of features left: 13

Feature selection: Univariate
Accuracy: 0.91 (+/- 0.04)
Number of features left: 5

Feature selection: Variance Threshold
Accuracy: 0.90 (+/- 0.04)
Number of features left: 21


# Logistic Regression 

In [65]:
logreg = linear_model.LogisticRegression(C=1e5)
clf_all(logreg, features, labels, domain)


Univariate - valuable features 

12. feature name: OFF
15. feature name: AST
17. feature name: BLK
22. feature name: W
23. feature name: H

Variance Threshold - removed 

5. feature name: FG%
8. feature name: 3P%
11. feature name: FT%

Feature selection: None
Accuracy: 0.89 (+/- 0.06)
Number of features left: 24

Feature selection: Domain
Accuracy: 0.90 (+/- 0.06)
Number of features left: 13

Feature selection: Univariate
Accuracy: 0.89 (+/- 0.06)
Number of features left: 5

Feature selection: Variance Threshold
Accuracy: 0.90 (+/- 0.05)
Number of features left: 21
